# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [ ]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [1]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [2]:
from ovs.indifference_zone import KN

In [3]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [4]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[ 6  5  6  7  7 27 32 33 34 35]
total reps	192
means		[0.83835058 2.39783421 2.76141715 4.4409697  4.66321696 6.00166801
 7.03259649 7.95113475 9.02445989 9.94936711]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [5]:
from ovs.fixed_budget import OCBA, OCBAM

In [6]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='min')

In [7]:
print(ocba)

OCBA(n_designs=10, budget=500, delta=5, n_0=5, obj=min)


call the `solve()` method to run the optimisation

In [8]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	0
allocations:	[217 187  47  13  11   5   5   5   5   5]
total reps:	500


NameError: name 'np' is not defined

## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [9]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocbam = OCBAM(model=environment, 
            n_designs=len(designs), 
            budget=2000, 
            delta=10, 
            n_0=5, 
            m=3,
            obj='max')

In [10]:
print(ocbam)

OCBA(n_designs=100, m=3, budget=2000, delta=10, n_0=5, obj=max)


In [11]:
results = ocba.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))
print('SEs:\t\t{0}'.format(ocba._ses))

NameError: name 'np' is not defined

In [ ]:
np.argsort(ocba._means)[-3:]

# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results